# CWA EBS - Run SQL Queries

In [99]:
import getpass
import time
import pandas as pd
import cx_Oracle
from IPython.display import Markdown
from ipywidgets import interact_manual, interact
from handy import run_query_df, display_df

# Stops warning about switching to SQL alchemy
import warnings
warnings.filterwarnings('ignore')

def nowstring():
    return time.strftime("%d/%m/%Y %H:%M:%S")

def run_queries(queries, titles, connstr, params):
    """Run multiple SQL queries and display their output"""
    display(Markdown("## Prameters: " + ",".join([str(p) for p in params])))
    display(Markdown("## Time: " + nowstring()))
    # If we have more queries than titles, pad the titles with ""
    titles += [""] * (len(queries) - len(titles))
    for title, query in zip(titles, queries):
        display(Markdown("### " + title))
        df = run_query_df(query, connstr, params)
        display_df(df)
    display(Markdown("## END " + nowstring()))

## Database connection details (CWA)

In [77]:
username = "query"
database = "host.docker.internal"
password = getpass.getpass("CWA password: ")
connstr = f"{username}/{password}@{database}:1571/cwa"

CWA password:  ········


## Define Queries, Headings and Parameters (and button for running them)

In [ ]:
vendor_sql = """
select
vendor_site_code_alt "Name",
vendor_site_id,
vendor_id,
address_line1,
attribute3 "Type"
from po.po_vendor_sites_all
where vendor_site_code = :x
"""

submissions_sql = """
select
v.vendor_site_code as "Account"
,v.vendor_site_code_alt as "Name"
,p.type_lookup_code
,p.segment1 as "PO Number"
,p.attribute4 as "Parent Contract"
,p.start_date
,p.end_date
,p.authorization_status
,p.attribute11
from 
po.po_headers_all p
,po.po_vendor_sites_all v
where 
p.vendor_site_id=v.vendor_site_id
and type_lookup_code = 'STANDARD'
and v.vendor_site_code = :x
"""

queries = [vendor_sql, submissions_sql]
titles = ["Vendor Details", "Vendor's Submissions"]
params = ["0C184Z"]

@interact_manual()        
def go():
    run_queries(queries, titles, connstr, params)


### Alternative with simple input field for account number that auto queries upon update
0G934M 0Y167H 0C015Q 0C024A

In [98]:
@interact(account="")
def go(account):
    run_queries(queries, titles, connstr, [account])

interactive(children=(Text(value='', description='account'), Output()), _dom_classes=('widget-interact',))